Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
# doing the import for numpy
import numpy as np


In [91]:
# Looking at the head of the dataFrame
print(df.shape)
df.head(5)

(23040, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [92]:
df.isnull().sum()

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         23040
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23040 entries, 0 to 23039
Data columns (total 21 columns):
BOROUGH                           23040 non-null object
NEIGHBORHOOD                      23040 non-null object
BUILDING_CLASS_CATEGORY           23040 non-null object
TAX_CLASS_AT_PRESENT              23039 non-null object
BLOCK                             23040 non-null int64
LOT                               23040 non-null int64
EASE-MENT                         0 non-null float64
BUILDING_CLASS_AT_PRESENT         23039 non-null object
ADDRESS                           23040 non-null object
APARTMENT_NUMBER                  5201 non-null object
ZIP_CODE                          23039 non-null float64
RESIDENTIAL_UNITS                 23039 non-null float64
COMMERCIAL_UNITS                  23039 non-null float64
TOTAL_UNITS                       23039 non-null float64
LAND_SQUARE_FEET                  22987 non-null object
GROSS_SQUARE_FEET                 23039 non-null floa

In [0]:
# Will be dropping the Apartment number colum and the ease_ment column because they have
# Lots of Null values
df = df.drop(['APARTMENT_NUMBER', 'EASE-MENT'], axis=1)

In [95]:
df.shape

(23040, 19)

In [0]:
# I will drop the null values in the Land square feet and year built and others
df = df.dropna()

In [97]:
df.shape

(22953, 19)

In [0]:
# writing a function that can then be applied to remove change the string to a integer
def changeStr(theStr):
  theStr = str(theStr)
  if theStr == "########":
    theStr = '0'

  theStr = theStr.strip()
  return int(theStr.replace(",", ""))

In [0]:
# I am going to change the Land_square_feet to a number from a string
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].apply(changeStr)

In [100]:
df['SALE_PRICE'].value_counts()

0          6905
10          199
800000      125
750000      121
650000      120
           ... 
228920        1
304200        1
76050         1
1091920       1
491496        1
Name: SALE_PRICE, Length: 3801, dtype: int64

In [101]:
# Creating the subset of the dataFrame to have only single family dwellings
df = df[df["BUILDING_CLASS_CATEGORY"] == "01 ONE FAMILY DWELLINGS"]
df.shape

(5061, 19)

In [102]:
# Filtering to just the sale prices to a 
# sale price that was more than 100 thousand and less than 2 million.
df = df[(df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000) ]
df.shape

(3151, 19)

In [103]:
# Do train/test split. Use data from January — March 2019 to train. 
# Use data from April 2019 to test.

# First will check to see if the sale_date is in DateTime format
print(df['SALE_DATE'].dtype)
df['SALE_DATE'].value_counts()

object


01/31/2019    78
03/29/2019    62
02/28/2019    58
01/15/2019    57
01/24/2019    56
              ..
04/14/2019     1
02/17/2019     1
03/17/2019     1
03/09/2019     1
03/30/2019     1
Name: SALE_DATE, Length: 91, dtype: int64

In [104]:
# Will change sale date to datetime format
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
myCutOff = pd.to_datetime('2019-03-31')
train = df[df['SALE_DATE'] <= myCutOff]
test = df[df['SALE_DATE'] > myCutOff]
print(train.shape, test.shape)
train['SALE_DATE'].value_counts()

(2507, 19) (644, 19)


2019-01-31    78
2019-03-29    62
2019-02-28    58
2019-01-15    57
2019-01-24    56
              ..
2019-01-01     2
2019-03-30     1
2019-03-17     1
2019-03-09     1
2019-02-17     1
Name: SALE_DATE, Length: 68, dtype: int64

In [105]:
# Looking to see what to do the one hot coding on
nonNumbers = train.describe(exclude=np.number)
nonNumbers

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2507,2507,2507,2507,2507,2507,2507,2507
unique,5,6,1,2,13,2497,11,68
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,33 BAILEY PLACE,A1,2019-01-31 00:00:00
freq,1204,2382,2507,2476,919,2,919,78
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-30 00:00:00


In [106]:
train['RESIDENTIAL_UNITS'].value_counts()

1.0    2476
0.0      31
Name: RESIDENTIAL_UNITS, dtype: int64

In [107]:
train['COMMERCIAL_UNITS'].value_counts()

0.0    2467
1.0      39
2.0       1
Name: COMMERCIAL_UNITS, dtype: int64

In [108]:
train['TOTAL_UNITS'].value_counts()

1.0    2436
2.0      39
0.0      31
3.0       1
Name: TOTAL_UNITS, dtype: int64

In [109]:
train['BOROUGH'].value_counts(dropna=False)

4    1204
5     662
3     398
2     242
1       1
Name: BOROUGH, dtype: int64

In [110]:
train['NEIGHBORHOOD'].value_counts()  

OTHER                 2382
FLUSHING-NORTH          77
FOREST HILLS            17
BOROUGH PARK            12
ASTORIA                 11
BEDFORD STUYVESANT       8
Name: NEIGHBORHOOD, dtype: int64

In [111]:

train['TAX_CLASS_AT_PRESENT'].value_counts()

1     2476
1D      31
Name: TAX_CLASS_AT_PRESENT, dtype: int64

In [0]:
# Picking out the features that I feel I will want to drop.
# These will contain the high cardinal features and those that may have NAN values
features_to_drop = ['ADDRESS', 'SALE_DATE']
target = "SALE_PRICE"

features = train.columns.drop(features_to_drop + [target])

In [0]:
# Getting the separate dataframe for each type
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [0]:
# doing the import for the category coder
import category_encoders as ce

In [0]:
# Doing the one hot encoder
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [116]:
# Looking at the head of the x_train after doing the one hot encoding
print(f"This is the shape of the x_train: {X_train.shape}")
print(f"This is the shape of the x_test: {X_test.shape}")
X_train.head()


This is the shape of the x_train: (2507, 48)
This is the shape of the x_test: (644, 48)


,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
44,1,0,0,0,0,1,0,0,0,0,0,1,1,0,5495,801,1,0,0,0,0,0,0,0,0,0,0,0,0,11230.0,1.0,0.0,1.0,6800,1325.0,1930.0,1,1,0,0,0,0,0,0,0,0,0,0
61,0,1,0,0,0,1,0,0,0,0,0,1,1,0,7918,72,0,1,0,0,0,0,0,0,0,0,0,0,0,11427.0,1.0,0.0,1.0,4000,2001.0,1940.0,1,0,1,0,0,0,0,0,0,0,0,0
78,0,0,1,0,0,1,0,0,0,0,0,1,1,0,4210,19,0,1,0,0,0,0,0,0,0,0,0,0,0,10461.0,1.0,0.0,1.0,3500,2043.0,1925.0,1,0,1,0,0,0,0,0,0,0,0,0
108,1,0,0,0,0,1,0,0,0,0,0,1,1,0,5212,69,0,1,0,0,0,0,0,0,0,0,0,0,0,11226.0,1.0,0.0,1.0,4000,2680.0,1899.0,1,0,1,0,0,0,0,0,0,0,0,0
111,1,0,0,0,0,1,0,0,0,0,0,1,1,0,7930,121,0,0,1,0,0,0,0,0,0,0,0,0,0,11203.0,1.0,0.0,1.0,1710,1872.0,1940.0,1,0,0,1,0,0,0,0,0,0,0,0


In [0]:
# Doing kBest to select the features 
# Doing some imports to start
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression



In [118]:
for i in range(1, len(X_train.columns)+1):

  print(f"The number of features selected {i}")
  selector = SelectKBest(score_func=f_regression, k=i);

  X_Train_selected = selector.fit_transform(X_train, y_train);
  X_Test_selected = selector.transform(X_test);



  # We will then check the mean absolute error for the model on the features
  # That are selected
  model = LinearRegression()
  model.fit(X_Train_selected, y_train)
  y_pred_train = model.predict(X_Train_selected);
  y_pred_test = model.predict(X_Test_selected);
  mae_train = mean_absolute_error(y_train, y_pred_train)
  mae_test = mean_absolute_error(y_test, y_pred_test)

  print(f"The mean abolute error for training is: ${mae_train:,.0f}")
  print(f"The mean abolute error for testing is: ${mae_test:,.0f}")
  print("\n")


The number of features selected 1
The mean abolute error for training is: $193,398
The mean abolute error for testing is: $183,641


The number of features selected 2
The mean abolute error for training is: $190,488
The mean abolute error for testing is: $179,555


The number of features selected 3
The mean abolute error for training is: $189,890
The mean abolute error for testing is: $179,291


The number of features selected 4
The mean abolute error for training is: $189,890
The mean abolute error for testing is: $179,291


The number of features selected 5
The mean abolute error for training is: $179,974
The mean abolute error for testing is: $170,483


The number of features selected 6
The mean abolute error for training is: $172,487
The mean abolute error for testing is: $169,982


The number of features selected 7
The mean abolute error for training is: $168,518
The mean abolute error for testing is: $168,140


The number of features selected 8
The mean abolute error for training

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

The mean abolute error for training is: $153,458
The mean abolute error for testing is: $156,394


The number of features selected 19
The mean abolute error for training is: $153,349
The mean abolute error for testing is: $156,255


The number of features selected 20
The mean abolute error for training is: $153,349
The mean abolute error for testing is: $156,255


The number of features selected 21
The mean abolute error for training is: $153,129
The mean abolute error for testing is: $154,396


The number of features selected 22
The mean abolute error for training is: $153,079
The mean abolute error for testing is: $154,426


The number of features selected 23
The mean abolute error for training is: $153,079
The mean abolute error for testing is: $154,426


The number of features selected 24
The mean abolute error for training is: $153,079
The mean abolute error for testing is: $154,426


The number of features selected 25
The mean abolute error for training is: $153,079
The mean abol

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

The mean abolute error for training is: $152,423
The mean abolute error for testing is: $154,788


The number of features selected 34
The mean abolute error for training is: $152,423
The mean abolute error for testing is: $154,781


The number of features selected 35
The mean abolute error for training is: $152,172
The mean abolute error for testing is: $154,760


The number of features selected 36
The mean abolute error for training is: $152,234
The mean abolute error for testing is: $154,593


The number of features selected 37
The mean abolute error for training is: $152,234
The mean abolute error for testing is: $154,593


The number of features selected 38
The mean abolute error for training is: $150,514
The mean abolute error for testing is: $155,529


The number of features selected 39
The mean abolute error for training is: $150,593
The mean abolute error for testing is: $155,697


The number of features selected 40
The mean abolute error for training is: $150,625
The mean abol

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal

The mean abolute error for training is: $150,379
The mean abolute error for testing is: $155,944




In [119]:
# I will choose 28 features because it gave the best mean absolute error for
# for the test set.  I am not sure why I am getting the the warnings in the output
selector = SelectKBest(score_func=f_regression, k=28)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# getting the column names
mask = selector.get_support()
columns = X_train.columns
selectedColumns = columns[mask]
selectedColumns


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


Index(['BOROUGH_3', 'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5',
       'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'NEIGHBORHOOD_FOREST HILLS', 'NEIGHBORHOOD_BOROUGH PARK',
       'TAX_CLASS_AT_PRESENT_1', 'BLOCK', 'BUILDING_CLASS_AT_PRESENT_A5',
       'BUILDING_CLASS_AT_PRESENT_A3', 'BUILDING_CLASS_AT_PRESENT_S1',
       'BUILDING_CLASS_AT_PRESENT_A6', 'BUILDING_CLASS_AT_PRESENT_A8',
       'BUILDING_CLASS_AT_PRESENT_S0', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET',
       'GROSS_SQUARE_FEET', 'BUILDING_CLASS_AT_TIME_OF_SALE_A5',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A3',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A6',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A8',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S0'],
      dtype='object')

In [0]:
# Trying some ridge Regression
from sklearn.linear_model import RidgeCV, Ridge
import matplotlib.pyplot as plt
import plotly.express as px


In [0]:
# the alphas that we are going to try
ourAlphas = [.001 ,.01, .1, .5, 100]

In [127]:
# I am going to put in the feature selected of 28 from the past kbest feature 
# selection
# Showing the alpha of .5 was the best
ridge = RidgeCV(alphas=ourAlphas)
ridge.fit(X_train_selected, y_train )
ridge.alpha_

0.5

In [164]:

# Trying the plotting of the Alpha that was the best (.5) types.


# Doing regularization
ridgeModel = Ridge(alpha=.5, normalize=True)
ridgeModel.fit(X_train_selected, y_train)
y_pred = ridgeModel.predict(X_test_selected)
  

  # finding the Mean absolute error
mae = mean_absolute_error(y_test, y_pred)

# Getting ready to plot the coefficients
coefficients = ridgeModel.coef_
# Putting them into a pandas series
theData = pd.Series(coefficients, selectedColumns)


print(f"The model using as alpha: {mAlpha}")
print(f"The mean absolute error for the alpha is: ${mae:.0f}")
print('The following are the coefficients')
# Sorting them and then plotting them



sorted = theData.sort_values()
px.bar(x=sorted, y =sorted.index, orientation="h" )




#theData.head()


The model using as alpha: 100
The mean absolute error for the alpha is: $160698
The following are the coefficients
